In [2]:
import pandas as pd
import numpy as np
import os

# load data
eee_root_dir = "C:\Capstone\Exp2 Facial Movements Individual Datafiles\[eee]"
eee_filepaths = os.listdir(eee_root_dir)
eee_dataframes = [pd.read_csv(os.path.join(eee_root_dir, path), skiprows=24) for path in eee_filepaths]
eee_all = pd.concat(eee_dataframes)

ugh_root_dir = "C:\Capstone\Exp2 Facial Movements Individual Datafiles\[ugh]"
ugh_filepaths = os.listdir(ugh_root_dir)
ugh_dataframes = [pd.read_csv(os.path.join(ugh_root_dir, path), skiprows=24) for path in ugh_filepaths]
ugh_all = pd.concat(ugh_dataframes)

# add label column to each df
eee_all.insert(loc=0, column='label', value='[eee]')
ugh_all.insert(loc=0, column='label', value='[ugh]')

# unify dataframes
all_data = pd.concat([eee_all, ugh_all])

In [ ]:
# import pandas as pd
# import numpy as np
# import os
#
# # Change the csv file paths as needed
# root_dir = "C:\Capstone\Exp2 Facial Movements Individual Datafiles"
# filepaths = os.listdir(root_dir)
# dataframes = [pd.read_csv(os.path.join(root_dir, path), skiprows=24) for path in filepaths]

In [3]:
# drop bad rows
#dataframes = [df[(df['Respondent Annotations active'].notna()) & (df['feature-x_1'] > 0) & (df['Respondent Annotations active'] != 'Happy; Surprise')] for df in dataframes]
all_data = all_data[(all_data['feature-x_1'].notna()) & (all_data['feature-x_1'] > 0)]

In [4]:
# Separate class label column and feature columns
# x_feature_columns = [x for x in dataframes[0].columns if 'feature-x' in x]
# y_feature_columns = [x for x in dataframes[0].columns if 'feature-y' in x]
x_feature_columns = [x for x in all_data.columns if 'feature-x' in x]
y_feature_columns = [x for x in all_data.columns if 'feature-y' in x]

# x_features = [df[x_feature_columns] for df in dataframes]
# y_features = [df[y_feature_columns] for df in dataframes]
#labels = [df['Respondent Annotations active'] for df in dataframes]
x_features = all_data[x_feature_columns]
y_features = all_data[y_feature_columns]
labels = all_data['label']

In [5]:
# Converting Pandas dataframes to Numpy nd-arrays
# x_arrays = [df.values for df in x_features]
# y_arrays = [df.values for df in y_features]
#label_columns = [df.values for df in labels]
x_array = x_features.values
y_array = y_features.values
label_column = labels.values

In [6]:
# Normalize each row in feature arrays, then concatenate x and y columns
# x_arrays = [array - np.mean(array, axis=1, keepdims=True) for array in x_arrays]
# y_arrays = [array - np.mean(array, axis=1, keepdims=True) for array in y_arrays]
# feature_arrays = [np.concatenate((xy[0], xy[1]), axis=1) for xy in zip(x_arrays, y_arrays)]
x_array = x_array - np.mean(x_array, axis=1, keepdims=True)
y_array = y_array - np.mean(y_array, axis=1, keepdims=True)
feature_array = np.concatenate((x_array, y_array), axis=1)

# Calculate mean and stdev vectors for each column in feature arrays
# feature_means = [np.mean(array, axis=0, keepdims=True) for array in feature_arrays]
# feature_stdev = [np.std(array, axis=0) for array in feature_arrays]
feature_means = np.mean(feature_array, axis=0, keepdims=True)
feature_stdev = np.std(feature_array, axis=0)

In [7]:
# Normalize each column in feature arrays
# feature_arrays = [(triple[0] - triple[1]) / triple[2] for triple in zip(feature_arrays, feature_means, feature_stdev)]
feature_array = (feature_array - feature_means) / feature_stdev

# Concatenate normalized feature arrays with label columns
# label_columns = [np.expand_dims(column, axis=1) for column in label_columns]
# complete_datasets = [np.concatenate(pair, axis=1) for pair in zip(label_columns, feature_arrays)]
label_column = np.expand_dims(label_column, axis=1)
complete_dataset = np.concatenate((label_column, feature_array), axis=1)

In [8]:
# Convert to dataframe and save to csv
save_dir = "C:\Capstone\Exp 2 Normalized"
# save_paths = [os.path.join(save_dir, path) for path in filepaths]
# dfs = [pd.DataFrame(array, columns=['label']+x_feature_columns+y_feature_columns) for array in complete_datasets]
# for pair in zip(dfs, save_paths):
#     pair[0].to_csv(pair[1], index=False)
save_path = os.path.join(save_dir, "all_data.csv")
all_data = pd.DataFrame(complete_dataset, columns=['label']+x_feature_columns+y_feature_columns)
all_data.to_csv(save_path, index=False)